# Lectura de Archivos

Librerias

In [1]:
import pandas as pd
pd.options.display.max_columns = None
# pd.options.display.max_rows = None
import os
from simpledbf import Dbf5
import pyodbc

PyTables is not installed. No support for HDF output.


Directorio de archivos, los almacenamos en una lista para iterar con el codigo

In [2]:
Path_ = r'path'
contenido = os.listdir(Path_)

file = []
for fichero in contenido:
    if os.path.isfile(os.path.join(Path_, fichero)) and fichero.endswith('.dbf'):
        file.append(fichero)
file

['marca.dbf', 'm_diesel.dbf', 'subdiesl.dbf', 'submarca.dbf', 't_maestr.dbf']

# Conexión a la BDD

Crear conexión a la BDD con Windows Authentication 

**Nota: idealmente se debe de agregra un try-excet**

In [3]:
server = 'Server'
conexion = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server,autocommit=True)

Crear conexión con User and Password

In [4]:
server = 'Server'
database = 'DataBase' 
username = 'User' 
password = 'Passwd' 

Corroboramos la conexión

In [5]:
try:
    cnxn = pyodbc.connect(
        'DRIVER={ODBC Driver 18 for SQL Server};'
        'SERVER='+server+';'
        'DATABASE='+database+';'
        'ENCRYPT=yes;UID='+username+';'
        'PWD='+ password
        )
    cursor = cnxn.cursor()
    cursor.close()
except:
    print("Error en conexión")

# Tabla a insertar el DBF

In [10]:
i = 2
df = Dbf5(Path_+file[i],codec='iso-8859-1').to_dataframe().fillna('')
df.head(1)

,MARCA,PANTALLA,BASEDATOS,POTMAX_RPM,RAL_FAB,GOB_FAB,VALIDA
0,100,3116 (170 H.P./2600rpm),101,2600,0,0,


Dado que son varias columnas se aplica la **ley del minimo esfuerzo**

Se utilizará una lista de lista donde obtenemos los valores del dataframe fila x fila 
para posteriormente iterar sobre ella 

__*Con esto esta bien*__

In [11]:
a,b = df.shape
lista = []
for i in range(a):
    lista_ = [] 
    for j in range(b):
        lista_.append(df.iloc[i][j])
    lista.append(lista_) 

**_Pero esto esta mejor_**

In [12]:
a,b = df.shape
df['VALIDA'] = df['VALIDA'].fillna('')
print('?,'*b)
lista = [tuple([str(df.iloc[i][j]) for j in range(b)]) for i in range(a)]
lista[:2]

?,?,?,?,?,?,?,


[('100', '3116 (170 H.P./2600rpm)', '101', '2600', '0', '0', ''),
 ('100', '3116 (185 H.P./2600rpm)', '102', '2600', '0', '0', '')]

In [14]:
sql = "EXEC [SCHEMA].[STORE_FOR_INSERT] ?,?,?,?,?,?,?"
for i in lista:
    try:
        cursor = cnxn.cursor()
        cursor.execute(sql,i)
        cnxn.commit()
        cursor.close()
    except:
        print("Error")
        cursor.close()